# 🤖 Telegram Bot для обработки видео

Этот блокнот запускает Telegram бота с поддержкой GPU для обработки видео.

**Важно:** Убедитесь, что включен GPU в настройках Colab:
- Меню → Среда выполнения → Сменить среду выполнения → Аппаратный ускоритель → GPU

## 📦 Установка зависимостей

In [ ]:
# Установка системных зависимостей
!apt-get update -qq
!apt-get install -y ffmpeg imagemagick

# Проверка GPU
!nvidia-smi

In [ ]:
# Установка Python зависимостей
!pip install --upgrade pip
!pip install python-telegram-bot==20.7
!pip install yt-dlp
!pip install moviepy
!pip install ffmpeg-python
!pip install google-api-python-client
!pip install google-auth-httplib2
!pip install google-auth-oauthlib
!pip install python-dotenv
!pip install Pillow
!pip install requests
!pip install asyncio
!pip install nest-asyncio

# Для работы с асинхронностью в Colab
import nest_asyncio
nest_asyncio.apply()

print("✅ Все зависимости установлены!")

## 🎤 Исправление Whisper для Colab

In [ ]:
# ИСПРАВЛЕНИЕ ПРОБЛЕМЫ С WHISPER В COLAB
print("🔧 Исправляем проблемы с Whisper в Colab...")

# Удаляем конфликтующие версии
print("1️⃣ Удаляем старые версии Whisper...")
!pip uninstall -y whisper openai-whisper faster-whisper whisper-jax -q

# Устанавливаем правильную версию OpenAI Whisper
print("2️⃣ Устанавливаем OpenAI Whisper...")
!pip install openai-whisper --upgrade --force-reinstall -q

# Проверяем установку
print("3️⃣ Проверяем установку...")
try:
    import whisper
    if hasattr(whisper, 'load_model'):
        print("✅ OpenAI Whisper установлен правильно!")
        
        # Предзагружаем модель для ускорения
        print("4️⃣ Предзагружаем модель 'base'...")
        model = whisper.load_model("base")
        print("✅ Модель 'base' загружена успешно!")
        print("🎉 Whisper готов к работе!")
        
    else:
        print("❌ Whisper установлен, но load_model недоступен")
        print("🔄 Пробуем альтернативный способ...")
        
        # Устанавливаем faster-whisper как альтернативу
        !pip install faster-whisper --upgrade -q
        
        from faster_whisper import WhisperModel
        model = WhisperModel("base")
        print("✅ faster-whisper установлен и готов к работе!")
        
except ImportError as e:
    print(f"❌ Не удалось импортировать whisper: {e}")
    print("🔄 Устанавливаем faster-whisper как альтернативу...")
    
    !pip install faster-whisper --upgrade -q
    
    try:
        from faster_whisper import WhisperModel
        model = WhisperModel("base")
        print("✅ faster-whisper установлен и готов к работе!")
    except Exception as e2:
        print(f"❌ Ошибка с faster-whisper: {e2}")
        print("⚠️ Субтитры могут не работать")

print("\n🎯 Whisper настроен для работы в Colab!")

## 📁 Клонирование репозитория

In [ ]:
import os
import shutil

# Удаляем папку если существует
if os.path.exists('/content/telegram-bot'):
    shutil.rmtree('/content/telegram-bot')

# Клонируем репозиторий (замените на ваш GitHub URL)
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO_NAME.git /content/telegram-bot

# Переходим в папку проекта
os.chdir('/content/telegram-bot')

print("📁 Репозиторий клонирован в:", os.getcwd())
print("📋 Файлы в проекте:")
!ls -la

## ⚙️ Настройка переменных окружения

In [ ]:
# Настройка переменных окружения
import os

# Токены и ключи
os.environ['TELEGRAM_BOT_TOKEN'] = '7850144731:AAHeHudyAVljC2J_CR8NLZznqnDHu8ZgLUw'
os.environ['GOOGLE_OAUTH_TOKEN_BASE64'] = 'gASV8QMAAAAAAACMGWdvb2dsZS5vYXV0aDIuY3JlZGVudGlhbHOUjAtDcmVkZW50aWFsc5STlCmBlH2UKIwFdG9rZW6UjP15YTI5LkEwQVMzSDZOeWlkcER0d3NEOExUbW9UTVpfdlFjWkh4SExFMEt1aGVNeFpNY01SM0Z4YnUzbk54SmpxbFFuaUg1YkZtSkxrdGJqZGVwQnRWTENJamp1aDMwYnZ4RFFUTU9sWHVUa250S0dSX1Eyb0NJWGJoTWdKMkNpbTJseDk2RTlOOEVTNUF6djJfa2VPVEVQMjJJSlBTSVcxb2VQWU5RMHgtdzZ3SWtSS002NHFmeGlZLVBfSEpFcEJXbUlHVTNURENkLUlxa2FDZ1lLQWJNU0FSQVNGUUhHWDJNaVN5RENEd1BTVU9hWDZFZlk2MTFHOFEwMjA2lIwGZXhwaXJ5lIwIZGF0ZXRpbWWUjAhkYXRldGltZZSTlEMKB+kHHgsUOAAAAJSFlFKUjBFfcXVvdGFfcHJvamVjdF9pZJROjA9fdHJ1c3RfYm91bmRhcnmUTowQX3VuaXZlcnNlX2RvbWFpbpSMDmdvb2dsZWFwaXMuY29tlIwZX3VzZV9ub25fYmxvY2tpbmdfcmVmcmVzaJSJjAdfc2NvcGVzlF2UjCVodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZllGGMD19kZWZhdWx0X3Njb3Blc5ROjA5fcmVmcmVzaF90b2tlbpSMZzEvLzBjczBxMUxUZVE4SUlDZ1lJQVJBQUdBd1NOd0YtTDlJcnV1THV3Z2xld3Y1RG1VN24zak5id3pVU0NudVF6d09IeTRXS2dBam9HUlNiMHNhaXBFVFVqcUQ5eDZMTk1DS29ySDCUjAlfaWRfdG9rZW6UTowPX2dyYW50ZWRfc2NvcGVzlF2UjCVodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZllGGMCl90b2tlbl91cmmUjCNodHRwczovL29hdXRoMi5nb29nbGVhcGlzLmNvbS90b2tlbpSMCl9jbGllbnRfaWSUjEg5MTQwNDc4ODM1ODItdTJtYjJsNDQ0Z2c1cGFzOG9pZ2ZvMHM4c2owYmpjYmUuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb22UjA5fY2xpZW50X3NlY3JldJSMI0dPQ1NQWC1DOTcxYW9tQndRZVItZzhXMFd1bTVpTUtfLWlMlIwLX3JhcHRfdG9rZW6UTowWX2VuYWJsZV9yZWF1dGhfcmVmcmVzaJSJjAhfYWNjb3VudJSMAJSMD19jcmVkX2ZpbGVfcGF0aJROdWIu'

# Создаем .env файл
env_content = f'''TELEGRAM_BOT_TOKEN={os.environ['TELEGRAM_BOT_TOKEN']}
GOOGLE_OAUTH_TOKEN_BASE64={os.environ['GOOGLE_OAUTH_TOKEN_BASE64']}
'''

with open('.env', 'w') as f:
    f.write(env_content)

print("✅ Переменные окружения настроены!")
print("📄 Создан файл .env")

## 🍪 Загрузка cookies файла

In [ ]:
# Загрузка cookies файла для yt-dlp
from google.colab import files
import os

print("🍪 Загрузите ваш cookies.txt файл для YouTube:")
print("ℹ️ Этот файл нужен для скачивания приватных или возрастных видео")
print("ℹ️ Если у вас нет cookies файла, просто пропустите этот шаг")

try:
    # Загружаем cookies файл
    uploaded = files.upload()
    
    # Ищем cookies файл среди загруженных
    cookies_file = None
    for filename in uploaded.keys():
        if 'cookies' in filename.lower() or filename.endswith('.txt'):
            cookies_file = filename
            break
    
    if cookies_file:
        # Переименовываем в стандартное имя
        if cookies_file != 'cookies.txt':
            os.rename(cookies_file, 'cookies.txt')
        
        print(f"✅ Cookies файл загружен: cookies.txt")
        
        # Добавляем путь к cookies в переменные окружения
        os.environ['COOKIES_FILE'] = '/content/telegram-bot/cookies.txt'
        
        # Обновляем .env файл
        with open('.env', 'a') as f:
            f.write(f'\nCOOKIES_FILE={os.environ["COOKIES_FILE"]}\n')
        
        print("📄 Путь к cookies добавлен в .env")
    else:
        print("⚠️ Cookies файл не найден среди загруженных")
        
except Exception as e:
    print(f"ℹ️ Cookies файл не загружен: {e}")
    print("ℹ️ Бот будет работать без cookies (ограниченная функциональность)")

## 📂 Создание необходимых папок

In [ ]:
# Создаем необходимые папки
import os

folders = ['temp', 'output']
for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"📁 Создана папка: {folder}")

print("✅ Все папки созданы!")

## 🧪 Тестирование импортов

In [ ]:
# Тестируем все импорты
try:
    import telegram
    print("✅ telegram импортирован")
    
    import yt_dlp
    print("✅ yt_dlp импортирован")
    
    import moviepy.editor as mp
    print("✅ moviepy импортирован")
    
    from google.oauth2.credentials import Credentials
    print("✅ google.oauth2 импортирован")
    
    from googleapiclient.discovery import build
    print("✅ googleapiclient импортирован")
    
    import asyncio
    print("✅ asyncio импортирован")
    
    print("\n🎉 Все библиотеки успешно импортированы!")
    
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")

## 🚀 Запуск бота

In [ ]:
# Запуск бота
import subprocess
import sys
import os

print("🤖 Запускаем Telegram бота...")
print("📍 Текущая директория:", os.getcwd())
print("📋 Файлы в директории:")
!ls -la

# Запускаем основной файл бота
try:
    # Если есть run_bot.py, используем его
    if os.path.exists('run_bot.py'):
        print("🔄 Запускаем через run_bot.py")
        !python run_bot.py
    # Иначе запускаем bot.py напрямую
    elif os.path.exists('bot.py'):
        print("🔄 Запускаем bot.py напрямую")
        !python bot.py
    else:
        print("❌ Не найден файл бота для запуска")
        print("📋 Доступные Python файлы:")
        !ls *.py
        
except KeyboardInterrupt:
    print("\n⏹️ Бот остановлен пользователем")
except Exception as e:
    print(f"❌ Ошибка при запуске бота: {e}")

## 📊 Просмотр логов

In [ ]:
# Просмотр логов бота
import os

if os.path.exists('bot.log'):
    print("📋 Последние 50 строк из bot.log:")
    !tail -50 bot.log
else:
    print("📋 Файл логов bot.log не найден")
    print("📁 Файлы в текущей директории:")
    !ls -la

## 🔧 Отладка и диагностика

In [ ]:
# Диагностическая информация
import sys
import os

print("🔍 Диагностическая информация:")
print(f"🐍 Python версия: {sys.version}")
print(f"📍 Рабочая директория: {os.getcwd()}")
print(f"🔑 TELEGRAM_BOT_TOKEN установлен: {'TELEGRAM_BOT_TOKEN' in os.environ}")
print(f"🔑 GOOGLE_OAUTH_TOKEN_BASE64 установлен: {'GOOGLE_OAUTH_TOKEN_BASE64' in os.environ}")

print("\n📦 Установленные пакеты:")
!pip list | grep -E "telegram|yt-dlp|moviepy|google"

print("\n💾 Использование диска:")
!df -h

print("\n🖥️ Использование памяти:")
!free -h

print("\n🎮 GPU информация:")
!nvidia-smi --query-gpu=name,memory.total,memory.used --format=csv

## 🔄 Ручной запуск (если автоматический не работает)

In [ ]:
# Ручной запуск бота с подробным выводом
import os
import sys

# Добавляем текущую директорию в путь Python
sys.path.insert(0, os.getcwd())

print("🔄 Попытка ручного запуска бота...")

try:
    # Импортируем и запускаем бота
    if os.path.exists('bot.py'):
        print("📥 Импортируем bot.py...")
        import bot
        print("✅ bot.py успешно импортирован")
        
        # Если есть функция main, запускаем её
        if hasattr(bot, 'main'):
            print("🚀 Запускаем функцию main()...")
            await bot.main()
        else:
            print("ℹ️ Функция main() не найдена в bot.py")
            
    else:
        print("❌ Файл bot.py не найден")
        
except Exception as e:
    print(f"❌ Ошибка при ручном запуске: {e}")
    import traceback
    traceback.print_exc()

## ⏹️ Остановка бота

In [ ]:
# Остановка всех процессов бота
import os
import signal

print("⏹️ Останавливаем все процессы бота...")

# Убиваем все Python процессы связанные с ботом
!pkill -f "python.*bot"
!pkill -f "python.*run_bot"

print("✅ Все процессы остановлены")